# Finding Optimal Resolution

Quick an dirty script that tries to automatically find the optimal resolution for testing some image

In [2]:
from src.analysis.grainpixels import pixels_per_grain
from skimage import io
import numpy as np
%load_ext autoreload
%autoreload 2

INPUT_IMAGE = '../data/test_combination/10HR/2153/trace.png'
img = io.imread(INPUT_IMAGE)

training_avg_r = 31

dim = 528
while True:
    _, avg_size, avg_r, _ = pixels_per_grain(img, (dim,dim))
    
    if np.abs(avg_r - training_avg_r) < 2:
        print(f"At Res {dim}x{dim}")
        print(f"Average Radius: {avg_r}\nAverage Size: {avg_size}")
        break
    elif avg_r > training_avg_r:
        dim -= 16
    elif avg_r < training_avg_r:
        dim += 16

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
At Res 640x640
Average Radius: 29.726749982747673
Average Size: 3417.7169811320755
